In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

from pathlib import Path
import json
from src.evaluation.ensembling import *
from src.utils.conver_to_coco import to_coco
import torch
import numpy as np



In [ ]:
# Confidence threshold to normalize the weights you need to prepare this beforehand
confidence_thr = {
    'predictions_faster_rcnn_resnet101.json' : 0.664,
    'predictions_retinanet_swint.json' : 0.401,
    'predictions_faster_rcnn_resnet50.json' :0.663,
    'predictions_yolov5_large.json' : 0.114,
    'predictions_yolov5_medium.json': 0.238,
    'predictions_efficientdet_d4.json' : 0.183
}
with open('confidence_thresholds.json', 'w') as  f:
    json.dump(confidence_thr, f)

In [ ]:
ann_path = Path('/datagrid/personal/kuntluka/dental_rtg3/annotations.json')

In [ ]:
#load predictions
pred_path = Path('ens_search')
preds = list(pred_path.iterdir())
preds = [pred.name for pred in preds]

with open('confidence_thresholds.json', 'r') as f:
    confidence_thr = json.load(f)
confidences = [confidence_thr[p] for p in preds]

preds_dicts = []
for pred_name in preds:
    with open(pred_path / pred_name, 'r') as f:
        preds_dicts.append(json.load(f))


with open('metadata4000.json', 'r') as f:
    meta_data = json.load(f)

In [ ]:
boxesensemble = BoxesEnsemble(meta_data, preds_dicts, ann_path, confidences )
boxesensemble.load_pred_eval(ann_path, meta_data)

In [ ]:
# In the casae you used optuna to find best ensembling parameters load the study and check the parameters
import optuna
study = optuna.load_study(
    storage="sqlite:///ens_search.db",
    study_name='ens_search_area'
)
study.best_trial

In [ ]:
# You need to manually set the parameters for ensembling (guess the numbers or do hyper-parameter search)
wbf_area_w = [[0.936,0.774, 0.84],[0.31,0.47,0.31],[0.98, 0.85,0.88], [0.72,0.69,0.91]]
wbf_t_area = 0.642

boxesensemble.evaluate_ensemble(wbf_area_w, wbf_t_area, 'test', ensemble_method='wbf', sigma=0.5)

In [ ]:
snms_argmax = [4, 0, 0, 0, 4, 5]
snms_w = [1, 0.12, 0.12, 0.12]
t =  0.7
sigma = 0.8

# boxesensemble.evaluate_ensemble([1,0.4, 0.85, 0.85], 0.65, 'test', ensemble_method='wbf')
boxesensemble.evaluate_ensemble(snms_w, t, 'test', ensemble_method='snms', sigma=sigma)

In [ ]:
nmw_indexes =[5, 1, 4, 5, 1]
nmw_w = [0.85, 0.25, 0.70, 0.85]
t =  0.45

boxesensemble.evaluate_ensemble(nmw_w, t, 'test', ensemble_method='nwm', sigma=sigma)

In [ ]:
nms_indexes = [6, 2, 2, 5, 4]

nms_w = [1, 0.4, 0.4, 0.85]
t =  0.6

boxesensemble.evaluate_ensemble(nms_w, t, 'test', ensemble_method='nms', sigma=sigma)
